In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# usa2 = pd.read_csv("../data/Annotations/GSE167202.csv", index_col=0)
# usa2[(usa2["covid_status"] == "negative") & (usa2["ed_disposition"] != "Discharged")] # negative but addmited to hospital ?
# usa2[(usa2["covid_status"].str.strip() == "positive") & (usa2["ed_disposition"].str.strip() == "Discharged")] # positive but not addmited to hospital

In [ ]:
df = pd.read_csv("../../data/raw/SampleSheetALL.csv", index_col=0)

In [ ]:
usa2 = pd.read_csv("../../data/Annotations/GSE167202.csv", index_col=0)
usa2.covid_status.unique()

In [ ]:
to_drop = usa2[
    (usa2["covid_status"].str.strip() == "negative")
    & (usa2["ed_disposition"].str.strip() != "Discharged")
].index  # negative but not discharged patients are removed!

In [ ]:
def descrbe(df):
    print(df.head())
    print(f"Shape: {df.shape}")
    print("\n")
    print(f"Is nan?: {df.isna().sum().any()}")
    print("\n")
    print(df.isna().sum())

    if df.ICU.isna().any():
        print(df[df.ICU.isna()])

    print("*" * 75)

In [ ]:
usa2 = pd.read_csv("../../data/Annotations/GSE167202.csv", index_col=0)
usa2 = usa2.drop(to_drop)
usa2["ed_disposition"].value_counts(dropna=False)

In [ ]:
# external annots

# PUM
pl = pd.read_csv("../../data/Annotations/PL-PUM.csv", index_col=0)
pl["Sample_Group"] = "COVID-19 PL"
pl["Status"] = "COVID-19 PL"

pl_annots = pd.read_csv(
    "../../data/raw/InitialData/Covid19-2-PUM/Annotations.csv", index_col=1
)[["P_e_ (M/K)", "Wiek"]]
pl = pd.concat((pl, pl_annots), axis=1)
pl["ICU"] = "non-ICU"
pl = pl[["Sample_Group", "Status", "P_e_ (M/K)", "Wiek", "ICU"]]
pl.columns = ["Sample_Group", "Status", "Sex", "Age", "ICU"]

# ES
es = pd.read_csv("../../data/Annotations/GSE168739.csv", index_col=0)
es["Sample_Group"] = "COVID-19 ES"
es["Status"] = "COVID-19 ES"
es = es[["age", "gender", "Sample_Group", "Status"]]
es.columns = ["Age", "Sex", "Sample_Group", "Status"]

epicovid = pd.read_excel(
    "../../data/Annotations/EPICOVID_GROUPS.xlsx",
    index_col=0,
    skiprows=0,
)
mapper = pd.read_csv("../../data/Annotations/GSE168739.csv", index_col=0).description
mapper = dict(zip(mapper.values, mapper.index))
epicovid.index = epicovid.index.map(mapper)
epicovid = epicovid[
    ["GROUP (G1: ICU; G2: Hospitalization with O2 support; G3: Mild, home)"]
]
epicovid.columns = ["ICU"]
epicovid.ICU = epicovid.ICU.map({"G1": "ICU", "G2": "non-ICU", "G3": "Home"})
es = pd.concat((es, epicovid), axis=1)

# USA 1
usa = pd.read_csv("../../data/Annotations/GSE174818.csv", index_col=0)
id_converter = dict(zip(usa["title"], usa.index))
usa = usa[["Sex", "age", "covid status"]]

usa_icu = pd.read_excel(
    "../../data/Annotations/GSE174818_supplement.xlsx", index_col=0, skiprows=1
)["ICU (1=yes)"]

usa_icu.index = usa_icu.index.map(id_converter)
usa_icu = usa_icu.loc[~usa_icu.index.isna()]
usa_icu = usa_icu.replace({1: "ICU", 0: "non-ICU"})

usa = pd.concat((usa, usa_icu), axis=1)
usa.columns = ["Sex", "Age", "Status", "ICU"]
usa["Sample_Group"] = "COVID-19 USA"

usa["Status"] = usa["Status"].map(
    {1: "COVID-19 USA 1", 0: "Other respiratory infections USA 1"}
)

# USA 2
usa2 = pd.read_csv("../../data/Annotations/GSE167202.csv", index_col=0)
usa2 = usa2.drop(to_drop)

usa2 = usa2[["covid_status", "Sex", "age", "ed_disposition"]]
usa2.columns = ["Status", "Sex", "Age", "ICU"]
usa2["Sample_Group"] = "COVID-19 USA 2"

usa2["ICU"] = (
    usa2["ICU"]
    .str.strip()
    .replace(
        {
            "Floor Admission": "non-ICU",
            "Discharged": "Home",
            "ICU": "ICU",
            "NA": np.NaN,
            "Death": "ICU",
        }
    )
)


usa2["Status"] = (
    usa2["Status"]
    .str.strip()
    .map(
        {
            "negative": "Healthy controls",
            "positive": "COVID-19 USA 2",
            "other infection": "Other respiratory infections USA 2",
        }
    )
)

usa2.loc[usa2.Status == "Healthy controls", "ICU"] = "Healthy controls"
usa2["Sex"] = usa2["Sex"].str.strip()

# hb controls
hb = pd.read_csv("../../data/Annotations/Sample_sheet_healthy_blood.csv", index_col=0)
hb = hb[["Age", "Sex"]]
hb["Sample_Group"] = "Healthy controls"
hb["ICU"] = "Healthy controls"
hb["Status"] = "Healthy controls"

In [ ]:
descrbe(usa2)

descrbe(pl)
descrbe(es)
descrbe(usa)
descrbe(hb)

In [ ]:
# concate
df = pd.concat((pl, es, usa, usa2, hb))
df

In [ ]:
df.ICU.value_counts(dropna=False)

In [ ]:
df.ICU[df.ICU.isna()]

In [ ]:
df.Sex.unique()

In [ ]:
df.Sex = df.Sex.str.strip().map(
    {"K": "F", "M": "M", "F": "F", "Male": "M", "Female": "F"}
)
df.Sex.unique()

In [ ]:
df.Status.value_counts()

In [ ]:
df.Status.isna().any()

In [ ]:
df.Sex.isna().any()

In [ ]:
df.to_csv("../../data/raw/SampleSheet.csv")